In [141]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import random

In [142]:
random.seed(42)
np.random.seed(42)

In [143]:
path_d = '/Users/bumsoojoe/Desktop/Section4_project/csv/diabetes_prediction_dataset.csv'
path_m = '/Users/bumsoojoe/Desktop/Section4_project/csv/medical.csv'

In [144]:
dliabetes = pd.read_csv(path_d)
medical = pd.read_csv(path_m)

In [145]:
df_d = dliabetes.drop_duplicates()
df_d = df_d.reset_index(drop=True)
df_d.rename(columns={'gender':'sex','smoking_history':'smoking'},inplace=True)
df_d['sex'] = df_d['sex'].str.lower()
df_d

,sex,age,hypertension,heart_disease,smoking,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80.0,0,1,never,25.19,6.6,140,0
1,female,54.0,0,0,No Info,27.32,6.6,80,0
2,male,28.0,0,0,never,27.32,5.7,158,0
3,female,36.0,0,0,current,23.45,5.0,155,0
4,male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,female,36.0,0,0,No Info,24.60,4.8,145,0
96142,female,2.0,0,0,No Info,17.37,6.5,100,0
96143,male,66.0,0,0,former,27.83,5.7,155,0
96144,female,24.0,0,0,never,35.42,4.0,100,0


In [146]:
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96146 entries, 0 to 96145
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sex                  96146 non-null  object 
 1   age                  96146 non-null  float64
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking              96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.6+ MB


In [147]:
df_d = df_d.astype({'age':'int64'})
df_d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96146 entries, 0 to 96145
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   sex                  96146 non-null  object 
 1   age                  96146 non-null  int64  
 2   hypertension         96146 non-null  int64  
 3   heart_disease        96146 non-null  int64  
 4   smoking              96146 non-null  object 
 5   bmi                  96146 non-null  float64
 6   HbA1c_level          96146 non-null  float64
 7   blood_glucose_level  96146 non-null  int64  
 8   diabetes             96146 non-null  int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 6.6+ MB


In [148]:
df_m = medical.copy()
df_m = df_m.drop_duplicates()
df_m = df_m.reset_index(drop=True)
df_m.rename(columns={'smoker':'smoking'},inplace=True)
df_m

,age,sex,bmi,children,smoking,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [149]:
other = list(df_d[df_d['sex'] == 'Other'].index)
df_d = df_d.drop(other)
df_d = df_d.reset_index(drop=True)
df_d

,sex,age,hypertension,heart_disease,smoking,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80,0,1,never,25.19,6.6,140,0
1,female,54,0,0,No Info,27.32,6.6,80,0
2,male,28,0,0,never,27.32,5.7,158,0
3,female,36,0,0,current,23.45,5.0,155,0
4,male,76,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
96141,female,36,0,0,No Info,24.60,4.8,145,0
96142,female,2,0,0,No Info,17.37,6.5,100,0
96143,male,66,0,0,former,27.83,5.7,155,0
96144,female,24,0,0,never,35.42,4.0,100,0


In [150]:
df_d['smoking']

0          never
1        No Info
2          never
3        current
4        current
          ...   
96141    No Info
96142    No Info
96143     former
96144      never
96145    current
Name: smoking, Length: 96146, dtype: object

In [151]:
nan = list(df_d[df_d['smoking'] == 'No Info'].index)
df_d = df_d.drop(nan)
df_d = df_d.reset_index(drop=True)

In [152]:
smoking_yes = ['ever','current']
smoking_no = ['never', 'former', 'not current']
def are_you_smoking(df,yes,no):
    new_df = df.copy()
    new_df = new_df.replace(yes,'yes')
    new_df = new_df.replace(no,'no')
    return new_df

In [153]:
df_d_n = are_you_smoking(df_d,smoking_yes,smoking_no)
df_d_n

,sex,age,hypertension,heart_disease,smoking,bmi,HbA1c_level,blood_glucose_level,diabetes
0,female,80,0,1,no,25.19,6.6,140,0
1,male,28,0,0,no,27.32,5.7,158,0
2,female,36,0,0,yes,23.45,5.0,155,0
3,male,76,1,1,yes,20.14,4.8,155,0
4,female,20,0,0,no,27.32,6.6,85,0
...,...,...,...,...,...,...,...,...,...
63254,female,26,0,0,no,34.34,6.5,160,0
63255,female,40,0,0,no,40.69,3.5,155,0
63256,male,66,0,0,no,27.83,5.7,155,0
63257,female,24,0,0,no,35.42,4.0,100,0


In [154]:
train_d, test_d = train_test_split(df_d_n, test_size=0.2, stratify= df_d_n['diabetes'], random_state=42)
train_d, val_d = train_test_split(train_d, test_size=0.2, stratify= train_d['diabetes'], random_state=42)

train_dy = train_d['diabetes']
train_dx = train_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

val_dy = val_d['diabetes']
val_dx = val_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

test_dy = test_d['diabetes']
test_dx= test_d.drop(columns=['diabetes','HbA1c_level','blood_glucose_level','hypertension','heart_disease'],axis=1)

print('train : ', train_dy.shape, train_dx.shape)
print('val : ', val_dy.shape, val_dx.shape)
print('test : ', test_dy.shape, test_dx.shape)

train :  (40485,) (40485, 4)
val :  (10122,) (10122, 4)
test :  (12652,) (12652, 4)


In [155]:
pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy="median"),
    XGBClassifier(
        objective="binary:logistic",
        eval_metric="error",
        n_estimators=200,
        random_state=42,
        n_jobs=-1,
        max_depth=7,
        learning_rate=0.1,
        use_label_encoder=False,
    ),
)
print(pipe)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric='error',
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=7, max_leaves=None,
                               min_child_weight=None, missing=n

/Users/bumsoojoe/anaconda3/envs/project/lib/python3.8/site-packages/xgboost/sklearn.py:1421: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


In [156]:
pipe.fit(train_dx, train_dy)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['sex', 'smoking'],
                                mapping=[{'col': 'sex', 'data_type': dtype('O'),
                                          'mapping': female    1
male      2
other     3
NaN      -2
dtype: int64},
                                         {'col': 'smoking',
                                          'data_type': dtype('O'),
                                          'mapping': no     1
yes    2
NaN   -2
dtype: int64}])),
                ('simpleimputer', SimpleImputer(strategy='median')),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbt...
                               feature_types=None, gamma=0, gpu_id=-1,
                               grow_policy='depthwise', importance_type=None,
                               interaction_constraints='', learning_rate=0.1,
                               max_bin=256, max_cat_threshold=64,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=7, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=200, n_jobs=-1, num_parallel_tree=1,
                               predictor='auto', random_state=42, ...))])

In [157]:
pipe.fit(train_dx, train_dy)

print("검증 정확도", pipe.score(val_dx, val_dy))

y_pred = pipe.predict(val_dx)
print(classification_report(val_dy, y_pred))

검증 정확도 0.8882631890930646
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      8996
           1       0.48      0.05      0.09      1126

    accuracy                           0.89     10122
   macro avg       0.69      0.52      0.52     10122
weighted avg       0.85      0.89      0.85     10122



In [158]:
df_m

,age,sex,bmi,children,smoking,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830
1333,18,female,31.920,0,no,northeast,2205.98080
1334,18,female,36.850,0,no,southeast,1629.83350
1335,21,female,25.800,0,no,southwest,2007.94500


In [159]:
df_m.columns

Index(['age', 'sex', 'bmi', 'children', 'smoking', 'region', 'charges'], dtype='object')

In [160]:
train_dx

,sex,age,smoking,bmi
34351,female,21,no,19.42
25451,male,49,yes,22.70
29065,male,48,yes,34.18
24755,male,66,yes,24.64
52348,female,50,no,56.35
...,...,...,...,...
32631,female,43,yes,27.32
57598,female,5,no,17.09
25712,female,71,no,28.17
59620,male,44,yes,28.45


In [172]:
df_m_d = df_m[['sex','age','smoking','bmi']]
df_m_d

,sex,age,smoking,bmi
0,female,19,yes,27.900
1,male,18,no,33.770
2,male,28,no,33.000
3,male,33,no,22.705
4,male,32,no,28.880
...,...,...,...,...
1332,male,50,no,30.970
1333,female,18,no,31.920
1334,female,18,no,36.850
1335,female,21,no,25.800


In [173]:
df_m_d['bmi'] = round(df_m_d['bmi'],2)

/var/folders/6z/x623xjv92hs04k54cpt4qhfw0000gn/T/ipykernel_3999/3208842265.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_m_d['bmi'] = round(df_m_d['bmi'],2)


In [174]:
df_m_d

,sex,age,smoking,bmi
0,female,19,yes,27.90
1,male,18,no,33.77
2,male,28,no,33.00
3,male,33,no,22.70
4,male,32,no,28.88
...,...,...,...,...
1332,male,50,no,30.97
1333,female,18,no,31.92
1334,female,18,no,36.85
1335,female,21,no,25.80


In [175]:
medical_pred = pipe.predict(df_m_d)
medical_pred

array([0, 0, 0, ..., 0, 0, 0])

In [180]:
df_m['diabetes'] = medical_pred
df_m

,age,sex,bmi,children,smoking,region,charges,diabetes
0,19,female,27.900,0,yes,southwest,16884.92400,0
1,18,male,33.770,1,no,southeast,1725.55230,0
2,28,male,33.000,3,no,southeast,4449.46200,0
3,33,male,22.705,0,no,northwest,21984.47061,0
4,32,male,28.880,0,no,northwest,3866.85520,0
...,...,...,...,...,...,...,...,...
1332,50,male,30.970,3,no,northwest,10600.54830,0
1333,18,female,31.920,0,no,northeast,2205.98080,0
1334,18,female,36.850,0,no,southeast,1629.83350,0
1335,21,female,25.800,0,no,southwest,2007.94500,0


In [182]:
train, test = train_test_split(df_m, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

train_y = train['charges']
train_x = train.drop(columns=['charges'],axis=1)

val_y = val['charges']
val_x = val.drop(columns=['charges'],axis=1)

test_y = test['charges']
test_x= test.drop(columns=['charges'],axis=1)

print('train : ', train_y.shape, train_x.shape)
print('val : ', val_y.shape, val_x.shape)
print('test : ', test_y.shape, test_x.shape)

train :  (855,) (855, 7)
val :  (214,) (214, 7)
test :  (268,) (268, 7)


In [184]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

#성능지표
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [190]:
pipe_l = make_pipeline(
    OrdinalEncoder(),
    XGBRegressor(),
)

print(pipe_l)

Pipeline(steps=[('ordinalencoder', OrdinalEncoder()),
                ('xgbregressor',
                 XGBRegressor(base_score=None, booster=None, callbacks=None,
                              colsample_bylevel=None, colsample_bynode=None,
                              colsample_bytree=None, early_stopping_rounds=None,
                              enable_categorical=False, eval_metric=None,
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_

In [191]:
pipe_l.fit(train_x, train_y)

y_train_pred = pipe_l.predict(train_x)
y_val_pred = pipe_l.predict(val_x)

In [193]:
def eval_models(y_true, y_pred) :
    mse = mean_squared_error(y_true, y_pred).round(3)
    rmse = np.sqrt(mse).round(3)
    mae = mean_absolute_error(y_true, y_pred).round(3)
    r2 = r2_score(y_true, y_pred)

    return mse, rmse, mae, r2

In [194]:
comparison = pd.DataFrame(index=['mse', 'rmse', 'mae', 'r2'], columns=['Train', 'Validation'])
comparison['Train'] = eval_models(train_y, y_train_pred)
comparison['Validation'] = eval_models(val_y, y_val_pred)

In [195]:
comparison

,Train,Validation
mse,303124.427000,3.177361e+07
rmse,550.567000,5.636809e+03
mae,337.801000,3.357795e+03
r2,0.997828,7.481705e-01


In [197]:
round(comparison,2)

,Train,Validation
mse,303124.43,31773614.72
rmse,550.57,5636.81
mae,337.80,3357.80
r2,1.00,0.75


In [205]:
y_test_pred = pipe_l.predict(test_x)
y_test_pred[2]

11424.43

In [203]:
test_y

899      8688.85885
1063     5708.86700
1255    11436.73815
298     38746.35510
237      4463.20510
           ...     
534     13831.11520
542     13887.20400
759      3925.75820
1283    47403.88000
1284     8534.67180
Name: charges, Length: 268, dtype: float64